<a href="https://colab.research.google.com/github/LearnByDoing2024/Youtube/blob/main/20241029_hugg_model_test/Copy_of_huggingface_model_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.17.0  # Ensure the correct version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 33.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [ ]:
import torch
from transformers import (
    T5Tokenizer, T5ForConditionalGeneration,
    DistilBertTokenizer, DistilBertForSequenceClassification,
    BertTokenizer, BertForQuestionAnswering,
    BartTokenizer, BartForConditionalGeneration,
    MarianMTModel, MarianTokenizer,
    AutoTokenizer, AutoModelForSeq2SeqLM  # Add these
)

In [ ]:
# 1. Text Generator (T5) - Initialize conversation
t5_tokenizer = T5Tokenizer.from_pretrained('t5-small')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
input_text = "Generate a short story about AI."
input_ids = t5_tokenizer.encode(input_text, return_tensors='pt')
gen_output = t5_model.generate(input_ids, max_length=200)
generated_text = t5_tokenizer.decode(gen_output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map

Generated Text:
Generate a short story about AI.


In [ ]:
# 2. Sentiment Analyzer (DistilBERT) - Analyze sentiment of generated text
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
sentiment_input_ids = distilbert_tokenizer.encode(generated_text, return_tensors='pt')
sentiment_output = distilbert_model(sentiment_input_ids)
sentiment = torch.argmax(sentiment_output.logits)
print("\nSentiment Analysis:")
print("Sentiment:", "Positive" if sentiment else "Negative")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1439: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_archive_file, map


Sentiment Analysis:
Sentiment: Positive


In [ ]:
# 3. Question Answering (BERT) - Ask & answer based on sentiment
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
if sentiment:
    question = "What is the main character's goal in this positive story?"
else:
    question = "What causes the conflict in this negative story?"

# Encode inputs with attention mask
encoding = bert_tokenizer.encode_plus(
    generated_text + " " + question,
    max_length=512,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

qa_input_ids = encoding['input_ids']
qa_attention_mask = encoding['attention_mask']

qa_output = bert_model(qa_input_ids, attention_mask=qa_attention_mask)

# Extract answer
answer_start = torch.argmax(qa_output.start_logits)
answer_end = torch.argmax(qa_output.end_logits) + 1
answer_ids = qa_input_ids[:, answer_start:answer_end]
qa_answer = bert_tokenizer.convert_tokens_to_string(bert_tokenizer.convert_ids_to_tokens(answer_ids[0]))

print("\nQuestion Answering:")
print("Question:", question)
print("Answer:", qa_answer)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a


Question Answering:
Question: What is the main character's goal in this positive story?
Answer: 


In [ ]:
# 4. Text Summarizer (Bart) - Summarize QA response
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
summarize_input_ids = bart_tokenizer.encode(qa_answer, return_tensors='pt')
summarize_output = bart_model.generate(summarize_input_ids, max_length=50)
summary = bart_tokenizer.decode(summarize_output[0], skip_special_tokens=True)
print("\nSummary:")
print(summary)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]


Summary:
CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Please submit your best shots of the U.S. for next week. Visit CNN.com/Travel next Friday for a new gallery of snapshots.


In [ ]:
# 5. Language Translator (MarianMT) - Translate summary to French
marian_model_name = "Helsinki-NLP/opus-mt-en-fr"
marian_model = MarianMTModel.from_pretrained(marian_model_name)
marian_tokenizer = MarianTokenizer.from_pretrained(marian_model_name)
translate_input_ids = marian_tokenizer.encode(summary, return_tensors='pt')
translate_output = marian_model.generate(translate_input_ids)
translated_summary = marian_tokenizer.decode(translate_output[0], skip_special_tokens=True)
print("\nTranslated Summary (French):")
print(translated_summary)

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/784k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]


Translated Summary (French):
CNN.com présentera des photos iReporter dans une galerie de photos de voyage hebdomadaire. Veuillez soumettre vos meilleures photos des États-Unis pour la semaine prochaine. Visitez CNN.com/Travel vendredi prochain pour une nouvelle galerie de photos.


In [ ]:
# 6. Chatbot (DialoGPT) - Respond to translated summary
from transformers import AutoTokenizer, AutoModelForCausalLM  # Update the import

dialogpt_tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
dialogpt_model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')
chat_input_ids = dialogpt_tokenizer.encode(translated_summary, return_tensors='pt')
chat_output = dialogpt_model.generate(chat_input_ids, max_length=100)
chat_response = dialogpt_tokenizer.decode(chat_output[0], skip_special_tokens=True)
print("\nChatbot Response:")
print(chat_response)

Downloading:   0%|          | 0.00/614 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/641 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/335M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Chatbot Response:
CNN.com présentera des photos iReporter dans une galerie de photos de voyage hebdomadaire. Veuillez soumettre vos meilleures photos des États-Unis pour la semaine prochaine. Visitez CNN.com/Travel vendredi prochain pour une nouvelle galerie de photos.com
